In [39]:
import pandas as pd

def load_data(file_path, chunksize=100000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunk['datetime'] = pd.to_datetime(chunk['datetime'])
        chunks.append(chunk)
    df = pd.concat(chunks, ignore_index=True)
    return df

def backtest_strategy(df, stop_loss_n=2, take_profit_m=2, price_step=5):
    trades = []
    sell_order = None
    daily_low = float('inf')  # 用于记录当日最低价
    last_price = None
    traded_levels = set()  # 记录已交易的整数价位

    for i in range(len(df)):
        current_price = df.loc[i, 'last_price']
        bid_price1 = df.loc[i, 'bid_price1']
        ask_price1 = df.loc[i, 'ask_price1']
        current_date = df.loc[i, 'datetime'].date()

        # 检查是否是新的一天
        if i == 0 or current_date != df.loc[i - 1, 'datetime'].date():
            daily_low = float('inf')  # 重置当日最低价
            sell_order = None
            traded_levels = set()  # 重置已交易的整数价位

        # 更新当日最低价（基于 bid_price1）
        if bid_price1 < daily_low:
            daily_low = bid_price1

        # 检查是否有新的低点突破并满足所有条件
        if bid_price1 % (price_step * 20) == 0 and bid_price1 == daily_low and bid_price1 not in traded_levels:
            traded_levels.add(bid_price1)

            # 卖出操作
            sell_price = bid_price1
            sell_order = bid_price1
            stop_loss_price = sell_price + stop_loss_n * price_step
            take_profit_price = sell_price - take_profit_m * price_step

            # 检查卖出是否成功
            if bid_price1 == sell_price:
                print(f"Sell order executed at {sell_price} at {df.loc[i, 'datetime']} (ask: {ask_price1}, bid: {bid_price1})")
                for j in range(i + 1, len(df)):
                    current_price = df.loc[j, 'last_price']
                    ask_price1 = df.loc[j, 'ask_price1']
                    bid_price1 = df.loc[j, 'bid_price1']
                    if current_price >= stop_loss_price:
                        print(f"Stop loss order executed at {ask_price1} at {df.loc[j, 'datetime']} (ask: {ask_price1}, bid: {bid_price1})")
                        trades.append({
                            'sell_price': sell_order,
                            'buy_to_cover_price': ask_price1,
                            'success': sell_order > ask_price1,
                            'profit': sell_order - ask_price1
                        })
                        break
                    elif current_price <= take_profit_price:
                        print(f"Take profit order executed at {bid_price1} at {df.loc[j, 'datetime']} (ask: {ask_price1}, bid: {bid_price1})")
                        trades.append({
                            'sell_price': sell_order,
                            'buy_to_cover_price': bid_price1,
                            'success': sell_order > bid_price1,
                            'profit': sell_order - bid_price1
                        })
                        break

        last_price = current_price

    return trades

def calculate_win_rate(trades):
    if not trades:
        return 0, 0, 0, 0
    total_trades = len(trades)
    wins = sum(1 for trade in trades if trade['success'])
    losses = sum(1 for trade in trades if trade['profit'] < 0)
    break_evens = total_trades - wins - losses
    win_rate = wins / total_trades
    return win_rate, total_trades, break_evens, losses, wins

def calculate_total_pnl(trades):
    total_pnl = sum(trade['sell_price'] - trade['buy_to_cover_price'] for trade in trades)
    return total_pnl

# 加载数据
df = load_data('future_taobao_ss2401_tick.csv')

# 运行回测，设定止损和止盈档位，每档为5点
stop_loss_n = 2  # 2档止损
take_profit_m = 2  # 2档止盈
price_step = 5  # 每档为5点
trades = backtest_strategy(df, stop_loss_n=stop_loss_n, take_profit_m=take_profit_m, price_step=price_step)

# 计算胜率
win_rate, total_trades, break_evens, losses, wins = calculate_win_rate(trades)

# 计算总盈亏
total_pnl = calculate_total_pnl(trades)

print(f"Total trades: {total_trades}")
print(f"Break-even trades: {break_evens}")
print(f"Losing trades: {losses}")
print(f"Winning trades: {wins}")
print(f"Win rate: {win_rate:.2%}")
print(f"Total PnL: {total_pnl}")


Sell order executed at 13800.0 at 2023-11-30 20:59:00.500000 (ask: 13805.0, bid: 13800.0)
Stop loss order executed at 13825.0 at 2023-11-30 21:01:59.500000 (ask: 13825.0, bid: 13820.0)
Sell order executed at 13800.0 at 2023-12-01 00:21:52.500000 (ask: 13805.0, bid: 13800.0)
Stop loss order executed at 13825.0 at 2023-12-01 00:25:09.500000 (ask: 13825.0, bid: 13815.0)
Sell order executed at 13700.0 at 2023-12-01 10:10:32.500000 (ask: 13705.0, bid: 13700.0)
Stop loss order executed at 13725.0 at 2023-12-01 10:31:07.500000 (ask: 13725.0, bid: 13720.0)
Sell order executed at 13700.0 at 2023-12-04 09:01:16 (ask: 13705.0, bid: 13700.0)
Stop loss order executed at 13725.0 at 2023-12-04 09:02:53 (ask: 13725.0, bid: 13720.0)
Sell order executed at 13600.0 at 2023-12-05 00:00:01 (ask: 13605.0, bid: 13600.0)
Take profit order executed at 13575.0 at 2023-12-05 00:59:40 (ask: 13580.0, bid: 13575.0)
Sell order executed at 13500.0 at 2023-12-05 09:03:17 (ask: 13505.0, bid: 13500.0)
Stop loss order ex

In [33]:
import pandas as pd

def load_data(file_path, chunksize=100000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunk['datetime'] = pd.to_datetime(chunk['datetime'])
        chunks.append(chunk)
    df = pd.concat(chunks, ignore_index=True)
    return df

def backtest_strategy(df, tick_interval=100, multiplier=100):
    trades = []
    sell_order = None
    daily_low = float('inf')  # 用于记录当日最低价
    last_price = None
    traded_levels = set()  # 记录已交易的整数价位

    for i in range(len(df)):
        current_price = df.loc[i, 'last_price']
        current_date = df.loc[i, 'datetime'].date()

        # 检查是否是新的一天
        if i == 0 or current_date != df.loc[i - 1, 'datetime'].date():
            daily_low = float('inf')  # 重置当日最低价
            sell_order = None
            traded_levels = set()  # 重置已交易的整数价位

        # 更新当日最低价
        if current_price < daily_low:
            daily_low = current_price

        # 打印所有的整数位日内价格新低点，确保每个价位在同一天内只记录一次
        if current_price % multiplier == 0 and current_price == daily_low and current_price not in traded_levels:
            print(f"New daily low at integer level {current_price} at {df.loc[i, 'datetime']}")
            traded_levels.add(current_price)

            # 卖出操作
            sell_price = current_price
            sell_order = current_price
            ask_price1 = df.loc[i, 'ask_price1']
            bid_price1 = df.loc[i, 'bid_price1']
            print(f"Sell order placed at {sell_price} at {df.loc[i, 'datetime']} (ask: {ask_price1}, bid: {bid_price1})")

            # 检查卖出是否成功
            print(f"Sell order executed at {sell_price} at {df.loc[i, 'datetime']} (ask: {ask_price1}, bid: {bid_price1})")
            buy_to_cover_idx = i + tick_interval
            if buy_to_cover_idx < len(df):
                buy_to_cover_price = df.loc[buy_to_cover_idx, 'ask_price1']
                buy_to_cover_bid_price1 = df.loc[buy_to_cover_idx, 'bid_price1']
                print(f"Buy to cover order executed at {buy_to_cover_price} at {df.loc[buy_to_cover_idx, 'datetime']} (ask: {buy_to_cover_price}, bid: {buy_to_cover_bid_price1})")
                trades.append({
                    'sell_price': sell_order,
                    'buy_to_cover_price': buy_to_cover_price,
                    'success': sell_order > buy_to_cover_price,
                    'profit': sell_order - buy_to_cover_price
                })
            else:
                print("Insufficient data for buy to cover order.")

        last_price = current_price

    return trades

def calculate_win_rate(trades):
    if not trades:
        return 0, 0, 0, 0
    total_trades = len(trades)
    wins = sum(1 for trade in trades if trade['success'])
    losses = sum(1 for trade in trades if trade['profit'] < 0)
    break_evens = total_trades - wins - losses
    win_rate = wins / total_trades
    return win_rate, total_trades, break_evens, losses, wins

def calculate_total_pnl(trades):
    total_pnl = sum(trade['sell_price'] - trade['buy_to_cover_price'] for trade in trades)
    return total_pnl

# 加载数据
df = load_data('future_taobao_ss2401_tick.csv')

# 运行回测，设定tick间隔为100
tick_interval = 30
trades = backtest_strategy(df, tick_interval=tick_interval)

# 计算胜率
win_rate, total_trades, break_evens, losses, wins = calculate_win_rate(trades)

# 计算总盈亏
total_pnl = calculate_total_pnl(trades)

print(f"Total trades: {total_trades}")
print(f"Break-even trades: {break_evens}")
print(f"Losing trades: {losses}")
print(f"Winning trades: {wins}")
print(f"Win rate: {win_rate:.2%}")
print(f"Total PnL: {total_pnl}")


New daily low at integer level 13800.0 at 2023-11-30 20:59:00.500000
Sell order placed at 13800.0 at 2023-11-30 20:59:00.500000 (ask: 13805.0, bid: 13800.0)
Sell order executed at 13800.0 at 2023-11-30 20:59:00.500000 (ask: 13805.0, bid: 13800.0)
Buy to cover order executed at 13815.0 at 2023-11-30 21:00:15 (ask: 13815.0, bid: 13805.0)
New daily low at integer level 13800.0 at 2023-12-01 00:21:52.500000
Sell order placed at 13800.0 at 2023-12-01 00:21:52.500000 (ask: 13805.0, bid: 13800.0)
Sell order executed at 13800.0 at 2023-12-01 00:21:52.500000 (ask: 13805.0, bid: 13800.0)
Buy to cover order executed at 13815.0 at 2023-12-01 00:22:08 (ask: 13815.0, bid: 13810.0)
New daily low at integer level 13700.0 at 2023-12-01 10:10:32.500000
Sell order placed at 13700.0 at 2023-12-01 10:10:32.500000 (ask: 13705.0, bid: 13700.0)
Sell order executed at 13700.0 at 2023-12-01 10:10:32.500000 (ask: 13705.0, bid: 13700.0)
Buy to cover order executed at 13705.0 at 2023-12-01 10:10:47.500000 (ask: 13